In [5]:
import numpy as np

class XGBoostRegressor:
    """
    Simple XGBoost-like regressor using regression trees as base learners.
    Only for educational purposes. Real XGBoost includes advanced optimizations!
    """
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, reg_lambda=1.0, gamma=0.0):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.reg_lambda = reg_lambda
        self.gamma = gamma
        self.trees = []
        self.gammas = []

    def fit(self, X, y):
        self.init_ = np.mean(y)
        y_pred = np.full(y.shape, self.init_)
        for _ in range(self.n_estimators):
            grad = y_pred - y  # gradient of squared loss
            hess = np.ones_like(y)  # hessian of squared loss = 1
            tree = RegressionTree(
                max_depth=self.max_depth, reg_lambda=self.reg_lambda, gamma=self.gamma
            )
            tree.fit(X, grad, hess)
            update = tree.predict(X)
            y_pred -= self.learning_rate * update
            self.trees.append(tree)
            self.gammas.append(self.learning_rate)

    def predict(self, X):
        y_pred = np.full((X.shape[0],), self.init_)
        for tree, gamma in zip(self.trees, self.gammas):
            y_pred -= gamma * tree.predict(X)
        return y_pred

class TreeNode:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class RegressionTree:
    """
    Regression tree for XGBoost, splits using gain criterion based on gradient and hessian sums.
    """
    def __init__(self, max_depth=3, reg_lambda=1.0, gamma=0.0):
        self.max_depth = max_depth
        self.reg_lambda = reg_lambda
        self.gamma = gamma
        self.root = None

    def fit(self, X, grad, hess):
        self.root = self._build_tree(X, grad, hess, depth=0)

    def _gain(self, G_left, H_left, G_right, H_right, G, H):
        def calc_score(G, H):
            return (G ** 2) / (H + self.reg_lambda)
        gain = (
            calc_score(G_left, H_left)
            + calc_score(G_right, H_right)
            - calc_score(G, H)
            - self.gamma
        )
        return gain

    def _build_tree(self, X, grad, hess, depth):
        if depth >= self.max_depth or len(X) <= 1:
            value = -np.sum(grad) / (np.sum(hess) + self.reg_lambda)
            return TreeNode(value=value)
        m, n = X.shape
        best_gain = -float("inf")
        best_idx, best_thr = None, None
        for feature in range(n):
            thresholds = np.unique(X[:, feature])
            for thresh in thresholds:
                left_mask = X[:, feature] <= thresh
                right_mask = ~left_mask
                if not np.any(left_mask) or not np.any(right_mask):
                    continue
                G_left = np.sum(grad[left_mask])
                H_left = np.sum(hess[left_mask])
                G_right = np.sum(grad[right_mask])
                H_right = np.sum(hess[right_mask])
                G = np.sum(grad)
                H = np.sum(hess)
                gain = self._gain(G_left, H_left, G_right, H_right, G, H)
                if gain > best_gain:
                    best_gain = gain
                    best_idx = feature
                    best_thr = thresh
        if best_gain <= 0 or best_idx is None:
            value = -np.sum(grad) / (np.sum(hess) + self.reg_lambda)
            return TreeNode(value=value)
        left_mask = X[:, best_idx] <= best_thr
        right_mask = ~left_mask
        left = self._build_tree(X[left_mask], grad[left_mask], hess[left_mask], depth + 1)
        right = self._build_tree(X[right_mask], grad[right_mask], hess[right_mask], depth + 1)
        return TreeNode(feature_index=best_idx, threshold=best_thr, left=left, right=right)

    def predict(self, X):
        return np.array([self._predict_row(x, self.root) for x in X])

    def _predict_row(self, x, node):
        while node.value is None:
            if x[node.feature_index] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.value

> ## Example usage:

In [6]:
X = np.array([[1], [2], [3], [4], [5], [6]])
y = np.array([1.2, 1.9, 3.0, 3.9, 5.1, 6.2])

xgb = XGBoostRegressor(n_estimators=10, learning_rate=0.1, max_depth=2)
xgb.fit(X, y)
preds = xgb.predict(np.array([[1.5], [3.5], [5.5]]))
print("XGBoost predictions:", preds)

XGBoost predictions: [5.36344276 3.32988688 1.6458851 ]
